In [ ]:
!pip install accelerate>=0.20.3 transformers

In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv(r'/content/train.csv',on_bad_lines='skip',engine='python')
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
data['toxic'].value_counts()

toxic
0    144277
1     15294
Name: count, dtype: int64

In [ ]:
data=data[['comment_text','toxic']]
data=data[0:1000]
data.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from tqdm import tqdm
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#model=model.to('cuda')

In [ ]:
sample_data=['i am eating apple','i am playing cricket']
tokenizer(sample_data,padding=True,truncation=True,max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 6207, 102], [101, 1045, 2572, 2652, 4533, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
X=list(data['comment_text'])
y=list(data['toxic'])


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [ ]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
#X_train_tokenized['input_ids'][0]

In [ ]:

len(X_train),len(X_val)

(800, 200)

In [ ]:
#create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [ ]:
len(X_train_tokenized["input_ids"])

800

In [ ]:
for key, val in X_train_tokenized.items():
    print(key, len(val))

input_ids 800
token_type_ids 800
attention_mask 800


In [ ]:
train_dataset[5]

{'input_ids': tensor([  101,  1000, 16948,  1024,  4790,  2005,  3972, 20624,  2239,  1013,
         18178,  5063,  6672, 27741,  2115,  2695,  2012,  1996,  2682, 21358,
          2094,  2006,  1037,  2047,  7761,  2396,  8572,  2015,  2033,  1012,
          2023,  2001,  1996,  2544,  2012,  1996,  2051,  1010,  2008,  2017,
          2241,  2115,  2695,  2006,  1012,  2017,  6866,  1024,  1000,  1000,
          1045,  2074,  3191,  1996,  3720,  1010,  1998,  2000,  2022,  3243,
          7481,  1010,  2514,  2008,  2009,  2515, 13225, 16948,  1005,  1055,
         11594,  1012,  1012,  1012,  2009,  2003,  2128,  6632,  6321, 23184,
          1012,  1012,  1012,  2144,  1045,  2903,  1996,  2025,  8010,  1997,
          1996,  3720,  2003,  8510,  1012,  1012,  1012,  2009,  2085,  2038,
          2340,  2981,  4216,  1012,  1998,  2000, 25416, 10421,  1996,  6685,
         13686,  2011,  6159,  1010,  2009,  2003,  4340,  2011,  1996, 10852,
          1054, 21030,  3120,  2008,  1

In [ ]:


def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


In [ ]:

trainer.train()

Step,Training Loss


TrainOutput(global_step=100, training_loss=0.2485567283630371, metrics={'train_runtime': 5283.6961, 'train_samples_per_second': 0.151, 'train_steps_per_second': 0.019, 'total_flos': 210488844288000.0, 'train_loss': 0.2485567283630371, 'epoch': 1.0})

In [ ]:

trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.21349173784255981,
 'eval_accuracy': 0.94,
 'eval_precision': 0.6956521739130435,
 'eval_recall': 0.7619047619047619,
 'eval_f1': 0.7272727272727272,
 'eval_runtime': 385.1151,
 'eval_samples_per_second': 0.519,
 'eval_steps_per_second': 0.065,
 'epoch': 1.0}

In [ ]:

np.set_printoptions(suppress=True)

In [ ]:

text = "That was good point"
# text = "go to hell"
#inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.0083, -1.1636]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.9598, 0.0402]], grad_fn=<SoftmaxBackward0>)


array([[0.9597641 , 0.04023586]], dtype=float32)

In [ ]:

trainer.save_model('CustomModel')